<table>
    <tr>
      <td>Introducción a
      </td>
      <td>
      <img src="https://media.licdn.com/dms/image/D5612AQF7GSp3l4pztQ/article-cover_image-shrink_720_1280/0/1686548640655?e=1715817600&v=beta&t=WQzv1EMkEEwZ0QZ0PF1anRKIHCl5BBH_YPZHdDQsWPM"  width=150/>
      </td>
     </tr>
</table>

## Regresión lineal

### Índice
[Obtención del modelo](#modelo)<br>
[Afinando el error](#error)<br>
[Intervalo de confianza](#intervalo)<br>
[Mejorando el modelo](#mejorando)<br>
[Lasso y Ridge](#lasso)<br>
[Regresión Series Temporales](#series)<br>


<a name="modelo"></a>
## Obtención del modelo

Empezamos cargando un fichero con las notas de las pruebas PISA. Recordamos la importancia del preprocesado, pero es aquí nos dan ya los datos preparados para que nos centremos en la regresión

In [ ]:
# Carga del fichero
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)
df

1.- Dividir las columnas en X e y en train y test. Nuestro objetivo será deducir la columna MAT desde SCI y REA

In [ ]:

XColumns = ["SCI", "REA"]
yColumn = "MAT"
X = df[XColumns]
y = df[yColumn]


In [ ]:
X

In [ ]:
y

2.- Dividir las columnas en X e y en train y test. Nuestro objetivo será deducir la columna MAT desde SCI y REA

In [ ]:

from sklearn.model_selection import train_test_split

test = 0.95 # 40%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

In [ ]:
X_train

In [ ]:
y_train

3.- Declarar el método y  entrenar con el conjunto train, obteniendo un *modelo*

In [ ]:
from sklearn.linear_model import LinearRegression
metodo = LinearRegression()
modelo = metodo.fit(X_train,y_train) # y  = f(x); MAT = b + a1 SCI + a2 REA

El modelo representa simplemente la recta que mejor ajusta las (X,y) dadas:

In [ ]:
modelo.intercept_, modelo.coef_

Es decir, dada una nota en ciencia `sci` y lectura `rea`, se puede obtener la nota de matemáticas `mat` de la siguiente forma:

mat = -56.25  + 0.90*sci + 0.21*rea    (ojo, estos números irán variando de ejecución e en ejecución)

Aunque veremos que estas predicciones las vamos a hacer automáticamente, podemos escribir por nuestra cuentra una función que las haga:

In [ ]:
def y_predict(X,modelo):
    s = modelo.intercept_
    for i,x in enumerate(X):
        s += x*modelo.coef_[i]
    return s

y_predict([400,450],modelo)

In [ ]:
modelo.predict([ [400,450], [600,580] ])

In [ ]:
df.describe()

Vamos a representar gráficamente los datos de entrenamiento y la recta modelo

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib

fig = plt.figure(figsize=(14,6))

# projection='3d' indica que este subplot es en 3d
ax = fig.add_subplot(1, 2, 1, projection='3d')
x = X_train["SCI"]
y = X_train["REA"]
z = y_train
ax.scatter3D( x, y, z)

y_1 = y_predict([300,300],modelo)
y_2 = y_predict([600,600],modelo)
ax.plot3D([300,600],[300,600],[y_1,y_2],color="green")
ax.set_zlabel(r"MAT", fontsize=10, color="blue")
ax.set_xlabel(r"SCI", fontsize=10, color="blue")
ax.set_ylabel(r"REA", fontsize=10, color="blue")
plt.show()

4.- Ahora predecimos con el test y mostramos el error

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print( f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}" )

In [ ]:
# bias

(y_test - y_pred).abs().mean()

In [ ]:
X_t = X_test.copy()
X_t["MAT"] = y_test
X_t["MAT_PRED"] = y_pred
X_t["RESIDUO"] = y_test - y_pred
X_t

In [ ]:
X_t.describe()

Y completamos la figura mostrando los puntos de entrenamiento, de test y la recta

In [ ]:
fig = plt.figure(figsize=(14,6))

# projection='3d' indica que este subplot es en 3d
ax = fig.add_subplot(1, 2, 1, projection='3d')
x = X_train["SCI"]
y = X_train["REA"]
z = y_train
ax.scatter3D( x, y, z,label="train")

y_1 = y_predict([300,300],modelo)
y_2 = y_predict([600,600],modelo)
ax.plot3D([300,600],[300,600],[y_1,y_2],color="green")
ax.set_zlabel(r"MAT", fontsize=10, color="blue")
ax.set_xlabel(r"SCI", fontsize=10, color="blue")
ax.set_ylabel(r"REA", fontsize=10, color="blue")


##### Esto es lo nuevo
x = X_test["SCI"]
y = X_test["REA"]
z = y_test
ax.scatter3D( x, y, z,label="test")
plt.legend()

plt.show()

**Ejercicio 1** Repetir desde el paso 1, cambiando tan solo el tamaño del test a 0.95 ¿qué sucede con el error?


Este es un ejemplo de overfitting: cuando por alguna razón (en este caso por falta de datos de entrenamiento), el modelo generado se comporta mucho mejor sobre el entrenamiento que sobre el test. Repitamos todo el proceso:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math

# 0 cargar datos
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1 Columnas x e y
XColumns = ["SCI", "REA"]
yColumn = "MAT"
X = df[XColumns]
y = df[yColumn]

# 2 train y test
test = 40 # 40%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3 crear model
metodo = LinearRegression()
modelo = metodo.fit(X_train,y_train) # y  = f(x); MAT = b + a1 SCI + a2 REA

# 4 evaluar
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
bias = (y_test - y_pred).mean()
print( f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}, BIAS: {round(bias,3)}" )


r^2: 0.948 RMSE: 11.313, MAE:9.1, BIAS: 3.154


<a name="error"></a>


## Afinando el error

**Ejercicio 2** Ejecutar varias veces el código anterior, se verá que se obtiene resultados diferentes ¿en qué punto del código se produce esta variación?

De hecho, si se quieren obtener resultados con una cierta verosimilitud tendremos que repetir el experimento (pasos 2,3,4) varias veces. Vamos a instalar el paquete progress bar para ver cómo progresan los experimentos

In [ ]:
# Carga del fichero
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math

from tqdm import tqdm

url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1
XColumns = ["SCI", "REA"]
yColumn = "MAT"
X = df[XColumns]
y = df[yColumn]

veces = 5000


resultados = []
for v in tqdm(range(veces)):
    # 2
    test = 0.4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

    # 3
    metodo = LinearRegression()
    modelo = metodo.fit(X_train,y_train)

    # 4
    y_pred = modelo.predict(X_test)
    r2 = r2_score(y_test,y_pred)
    rmse = math.sqrt(mean_squared_error(y_test,y_pred))
    mae = mean_absolute_error(y_test,y_pred)
    bias = (y_test - y_pred).mean()
    resultados.append([round(r2,3),round(rmse,3),round(mae,3),round(bias,3)])
    #print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

df_errores = pd.DataFrame(resultados,columns=["r^2","RMSE","MAE","BIAS"])
df_errores

100%|██████████| 5000/5000 [00:35<00:00, 138.99it/s]


,r^2,RMSE,MAE,BIAS
0,0.948,11.029,8.990,0.236
1,0.929,11.179,9.106,0.267
2,0.967,9.025,7.428,3.611
3,0.941,12.630,10.517,5.286
4,0.910,14.823,12.468,-0.202
...,...,...,...,...
4995,0.930,11.771,9.264,0.174
4996,0.965,10.061,8.120,0.809
4997,0.940,12.735,10.542,-0.141
4998,0.933,12.250,10.047,-4.880


In [ ]:
df_errores.describe()

,r^2,RMSE,MAE,BIAS
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.942336,11.708616,9.440680,0.015820
std,0.016781,1.208419,1.132656,2.956029
min,0.835000,7.544000,5.863000,-10.326000
25%,0.933000,10.896750,8.660000,-1.983250
50%,0.945000,11.729500,9.418500,-0.013500
75%,0.954000,12.505000,10.180750,1.992750
max,0.981000,17.515000,14.004000,11.321000


In [ ]:
df_errores.describe().loc["mean"]

r^2      0.942336
RMSE    11.708616
MAE      9.440680
BIAS     0.015820
Name: mean, dtype: float64

Estamos cometiendo un error +- 24. Paso final: generar el verdadero modelo

In [ ]:
metodo = LinearRegression()
modelo = metodo.fit(X,y)

In [ ]:
from joblib import dump
dump(modelo, 'predice_pisa.joblib')

['predice_pisa.joblib']

Es fácil hacer una función que haga el trabajo de los experimentos. Podemos usarla para evaluar la regresión con otros datos

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
from tqdm import tqdm

def evalua_regresion(df,XColumns,yColumn,veces=500):
    # 1
    X = df[XColumns]
    y = df[yColumn]



    resultados = []
    for v in tqdm(range(veces)):
        # 2
        test = 0.4
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

        # 3
        metodo = LinearRegression()
        modelo = metodo.fit(X_train,y_train)

        # 4
        y_pred = modelo.predict(X_test)
        r2 = r2_score(y_test,y_pred)
        rmse = math.sqrt(mean_squared_error(y_test,y_pred))
        mae = mean_absolute_error(y_test,y_pred)
        bias = (y_test - y_pred).mean()
        resultados.append([round(r2,3),round(rmse,3),round(mae,3),round(bias,3)])
        #print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

    df_errores = pd.DataFrame(resultados,columns=["r^2","RMSE","MAE","BIAS"])
    return df_errores.describe().loc["mean"]


In [ ]:
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1
XColumns = ["SCI", "REA"]
yColumn = "MAT"
print(evalua_regresion(df,XColumns,yColumn))

100%|██████████| 500/500 [00:02<00:00, 195.45it/s]

r^2      0.940588
RMSE    11.800504
MAE      9.493896
BIAS    -0.480132
Name: mean, dtype: float64


**Ejercicio 3**

- Si se en lugar de predecir "MAT" desde "SCI" y "REA" tuviéramos que predecir "SCI" desde las otras dos o "REA" desde las otras dos cual daría mejores resultados
- Si solo queremos utilizar un atributo, ya sea "SCI" o "REA" para predecir "MAT" ¿cuál usarías?

In [ ]:
from sklearn.preprocessing import MinMaxScaler

url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1
XColumns = ["MAT", "SCI","RPC"]
yColumn = "REA"

# escalado; inútil en este caso, pero por probar...
metodo_escalado  = MinMaxScaler()
modelo_escalado = metodo_escalado.fit(df[XColumns])
datos_escalados = modelo_escalado.transform(df[XColumns])

df.loc[:,XColumns] = datos_escalados

print(evalua_regresion(df,XColumns,yColumn))

100%|██████████| 500/500 [00:03<00:00, 130.15it/s]

r^2      0.906082
RMSE    13.673960
MAE     10.768182
BIAS    -0.403398
Name: mean, dtype: float64


In [ ]:
datos_escalados

array([[0.36135072, 0.42684625, 0.05929717],
       [0.1349085 , 0.19695949, 0.07953286],
       [0.34391345, 0.44917943, 0.12656474],
       [0.70276044, 0.79645604, 0.37645065],
       [0.71479531, 0.72966728, 0.37438062],
       [0.75810454, 0.76075743, 0.34440605],
       [0.20875293, 0.30831725, 0.08313502],
       [0.3067872 , 0.39283206, 0.09511507],
       [0.46278221, 0.45083258, 0.26879319],
       [0.77545513, 0.76047699, 0.37199276],
       [0.81115043, 0.90452179, 0.22692007],
       [0.77540988, 0.88874718, 0.32879124],
       [0.69863422, 0.7293998 , 0.32170915],
       [0.53249239, 0.55011278, 0.18792378],
       [0.63059897, 0.64799213, 0.20676588],
       [0.67796845, 0.63228495, 0.3921877 ],
       [0.74430932, 0.76332692, 0.60247425],
       [0.55865062, 0.55750239, 0.17353957],
       [0.6537469 , 0.70817676, 0.19144445],
       [0.33964346, 0.37542484, 0.11913222],
       [0.38155092, 0.35579289, 0.10336256],
       [0.78038224, 0.79011896, 0.41104609],
       [0.

<a name="intervalo"></a>
## Intervalos de confianza

Si ahora ejecutamos varias veces el código veremos que los resultados son bastante estables. Aun así persiste una duda...cuando hagamos una predicción ¿podemos estimar lo lejos que está del valor real?

Vamos a poder hacerlo con un par de condiciones:

1.- Que el bias sea muy próximo a 0

2.- Que el RMSE sea una normal

La condición 1 la tenemos; hay que tener en cuenta que un valor de alrededor de -0.1...0.1 en

In [ ]:
df.MAT.mean()

es muy pequeño. En cuanto al RMSE por simplificar nos conformamos con "ver" el histograma

In [ ]:
import matplotlib.pyplot as plt
#fig, ax = plt.subplots(figsize=(5, 5))
df_errores.RMSE.hist(bins=15)
plt.show()

Asumiendo que esto es una normal, podemos decir que, con un 95% de probabilidades, para cualquier predicción p el valor real estará en el intervalo

[p -2RMSE, p+2RMSE]

Este intervalo de confianza se mantiene siempre y cuando estemos (aprox.) dentro del rango de valores usados en el entrenamiento

In [ ]:
X_train.describe()

**Ejemplo**

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1
XColumns = ["SCI", "REA"]
yColumn = "MAT"
X = df[XColumns]
y = df[yColumn]

metodo = LinearRegression()
modelo = metodo.fit(X.values,y)

p = modelo.predict([[400,450]])[0]

p

Como el RMSE es aproximadamente 11.75, tendremos que con un 95% de probabilidades el valor real está entre

In [ ]:
RMSE = 11.75
(p-1.96*RMSE,p+1.96*RMSE)

In [ ]:
import matplotlib.pyplot as plt
df[["MAT"]].hist()
plt.plot([p-1.96*RMSE,p-1.96*RMSE],[0,1],color="pink",linewidth=5)
plt.plot([p+1.96*RMSE,p+1.96*RMSE],[0,1],color="pink",linewidth=5)
plt.plot([p,p],[0,1],color="yellow",linewidth=5)
plt.show()

Sin embargo, los siguientes diagramas de residuos nos indican que algo no está funcionando como esperábamos

In [ ]:
X = df[XColumns]
y = df[yColumn]

x = range(len(y))
y_pred = modelo.predict(X.values)
plt.scatter(x,y_pred,color="red",s=1)
plt.scatter(x,y,color="blue",s=8)
for i,v in enumerate(y_pred):
    plt.plot([x[i],x[i]], [v-2*RMSE,v+2*RMSE],color="green")
#for y_v in y_pred:

In [ ]:
x = range(len(y))
fig, ax = plt.subplots(figsize=(15, 5))
y_pred = modelo.predict(X.values)
ci = 1.96*RMSE
for i,v in enumerate(y_pred):
    plt.plot([x[i],x[i]], [v,y[i]],color="green")
ax.fill_between(x, ( y_pred-ci), ( y_pred+ci), color='b', alpha=.1)
ax.scatter(x,y_pred,color="red",s=8)
ax.scatter(x,y,color="blue",s=8)
plt.show()

In [ ]:
y_pred = metodo.predict(X.values)

x_plot = plt.scatter(y_pred, (y_pred - y), c='b')
plt.hlines(y=0, xmin= -1, xmax=800)

plt.hlines(y=2*RMSE, xmin= -1, xmax=800,color="r")
plt.hlines(y=-2*RMSE, xmin= -1, xmax=800,color="r")
plt.show()

<a name="mejorando"></a>
## Mejorando el modelo

El histograma nos sugiere que quizás el modelo sea mejor si dividimos el conjunto en dos

In [ ]:
df2 = df[df[yColumn]<450]
X = df2[XColumns]
y = df2[yColumn]

metodo = LinearRegression()
modelo = metodo.fit(X.values,y)

r2,RMSE,mae,bias = evalua_regresion(df2,XColumns,yColumn)
print(RMSE)

x = range(len(y))
y_pred = modelo.predict(X.values)
plt.scatter(x,y_pred,color="red",s=1)
plt.scatter(x,y,color="blue",s=8)
for i,v in enumerate(y_pred):
    plt.plot([x[i],x[i]], [v-2*RMSE,v+2*RMSE],color="green")

In [ ]:
x_plot = plt.scatter(y_pred, (y_pred - y), c='b')
plt.hlines(y=0, xmin= -1, xmax=800)

plt.hlines(y=2*RMSE, xmin= -1, xmax=800,color="r")
plt.hlines(y=-2*RMSE, xmin= -1, xmax=800,color="r")
plt.show()

In [ ]:
df2 = df[df[yColumn]>=450]
X = df2[XColumns]
y = df2[yColumn]

metodo = LinearRegression()
modelo = metodo.fit(X.values,y)

r2,RMSE,mae,bias = evalua_regresion(df2,XColumns,yColumn)
print(RMSE)

x = range(len(y))
y_pred = modelo.predict(X.values)
plt.scatter(x,y_pred,color="red",s=1)
plt.scatter(x,y,color="blue",s=8)
for i,v in enumerate(y_pred):
    plt.plot([x[i],x[i]], [v-2*RMSE,v+2*RMSE],color="green")

In [ ]:
x_plot = plt.scatter(y_pred, (y_pred - y), c='b')
plt.hlines(y=0, xmin= -1, xmax=800)

plt.hlines(y=2*RMSE, xmin= -1, xmax=800,color="r")
plt.hlines(y=-2*RMSE, xmin= -1, xmax=800,color="r")
plt.show()

<a name="lasso"></a>
##  Ridge y Lasso

Muy útil cuando hay pocos valores para entrenar

In [ ]:
# Carga del fichero
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)

# 1
XColumns = ["SCI", "REA"]
yColumn = "MAT"
X = df[XColumns]
y = df[yColumn]

# 2
from sklearn.model_selection import train_test_split
test = 0.95
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3
from sklearn.linear_model import LinearRegression
metodo = LinearRegression()
modelo = metodo.fit(X_train,y_train)

# 4
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

In [ ]:
from sklearn.linear_model import Lasso
modelo = Lasso(alpha=6).fit(X_train, y_train)
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

In [ ]:
from sklearn.linear_model import Ridge
modelo = Ridge(alpha=6).fit(X_train, y_train)
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

ElasticNet combina los dos

In [ ]:
from sklearn.linear_model import ElasticNet
modelo = ElasticNet(alpha=6, l1_ratio=0.5).fit(X_train, y_train)
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

<a name="series"></a>
##  Series temporales

El principal problema es preparar los datos para que coincidan el "futuro" que queremos


In [ ]:
url = "https://raw.githubusercontent.com/RafaelCaballero/BME/main/data/currencies.csv"
df_cur = pd.read_csv(url)[["Close_CAD","Close_JPY","Close_EUR"]]
df_cur

In [ ]:
df_cur.describe()

In [ ]:
import pandas as pd
def estadísticas(df):
  # solo las columnas numéricas
  df2 = df.select_dtypes(include=["number"])
  datos = []
  for c in df2:
      variable = df2[c]
      datos.append([variable.mean(), variable.median(), variable.std(), (variable-variable.median()).abs().median()])

  estad = pd.DataFrame(datos,columns=["mean","median","std","MAD"],index=df2.columns)
  return estad

estadísticas(df_cur)

Primer intento:

In [ ]:

def regresión(df,XColumns,yColumn):
  # 1
  X = df[XColumns]
  y = df[yColumn]

  # 2
  from sklearn.model_selection import train_test_split
  test = 0.70
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

  # 3
  from sklearn.linear_model import LinearRegression
  metodo = LinearRegression()
  modelo = metodo.fit(X_train,y_train)

  # 4
  from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
  import math
  y_pred = modelo.predict(X_test)
  r2 = r2_score(y_test,y_pred)
  rmse = math.sqrt(mean_squared_error(y_test,y_pred))
  mae = mean_absolute_error(y_test,y_pred)
  print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")


XColumns = ["Close_CAD", "Close_JPY"]
yColumn = "Close_EUR"
regresión(df_cur,XColumns,yColumn)

In [ ]:
import matplotlib.pyplot as plt
x = range(len(y))
fig, ax = plt.subplots(figsize=(15, 5))
y_pred = modelo.predict(X.values)
ci = 1.96*rmse
for i,v in enumerate(y_pred):
    plt.plot([x[i],x[i]], [v,y[i]],color="green",alpha=0.1)
ax.fill_between(x, ( y_pred-ci), ( y_pred+ci), color='b', alpha=.1)
ax.scatter(x,y_pred,color="red",s=1, label="predicho",alpha=0.5)
ax.scatter(x,y,color="blue",s=1,label="real",alpha=0.5)
plt.legend()
plt.show()

Sin embargo esto está mal y por varias razones:

1 Estamos calculando el cierre del mismo día...intentamos predecir el dato que ya tenemos

2 Mezclamos futuros y pasados (solución: TimeSeriesSplit())

In [ ]:
futuro = 200 #  días
euro_f = df_cur.loc[futuro:,"Close_EUR"]
euro_f, df_cur.Close_EUR


In [ ]:
euro_f2  = euro_f.reset_index(drop=True)
euro_f2

In [ ]:
df_cur["label"] = euro_f2
df_cur

Tenemos que quitar los nulos; ponemos todo junto

In [ ]:
futuro = 2 #  días
euro_f = df_cur.loc[futuro:,"Close_EUR"]
euro_f, df_cur.Close_EUR
euro_f2  = euro_f.reset_index(drop=True)
df_cur["label"] = euro_f2

XColumns = ["Close_CAD", "Close_JPY"]
yColumn = "label"
regresión(df_cur.dropna(),XColumns,yColumn)

In [ ]:
XColumns = ["Close_CAD", "Close_JPY", "Close_EUR"]
yColumn = "label"
df = df_cur.dropna()
regresión(df,XColumns,yColumn)

Esto tiene muy buena pinta, pero debemos comparar con la predicción más simple: la predicción naïve

In [ ]:
y_pred = df.Close_EUR
y_test = df.label
r2 = r2_score(y_test,y_pred)
rmse = math.sqrt(mean_squared_error(y_test,y_pred))
mae = mean_absolute_error(y_test,y_pred)
print(f"r^2: {round(r2,3)} RMSE: {round(rmse,3)}, MAE:{round(mae,3)}")

La predicción no era tan buena como pensábamos...